In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow_datasets as tfds
import tensorflow as tf
import os
import numpy as np 

In [20]:
# ind = tf.keras.Input(shape=(4), dtype=tf.int64)       # (None, 4)

# x = tf.keras.Input(shape=(35, 7), dtype=tf.float32)   # (None, 35, 7)
# net = tf.keras.layers.Dense(11, activation='relu')(x)   # (None, 35, 11)
# net = tf.reduce_max(net, axis=1, name='maxpool')          # (None, 11)

# dense_shape = np.array([2, 7, 8, 9, 11], np.int64)
# y = tf.scatter_nd(ind, net, dense_shape)

# model = tf.keras.Model(inputs=[x, ind], outputs=y)

In [22]:
# model.summary()

In [10]:
strategy = tf.distribute.MirroredStrategy()
print('Number of devices: {}'.format(strategy.num_replicas_in_sync))

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
Number of devices: 1


In [12]:
# # You can also do info.splits.total_num_examples to get the total
# # number of examples in the dataset.

# BUFFER_SIZE = 10000
# BATCH_SIZE_PER_REPLICA = 64
# BATCH_SIZE = BATCH_SIZE_PER_REPLICA * strategy.num_replicas_in_sync
# print('Batch size: {}'.format(BATCH_SIZE))


# datasets, info = tfds.load(name='mnist', with_info=True, as_supervised=True)
# mnist_train, mnist_test = datasets['train'], datasets['test']

# train_dataset = mnist_train.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
# eval_dataset = mnist_test.batch(BATCH_SIZE)

Batch size: 64


In [13]:
# mnist_train


<_OptionsDataset shapes: ((28, 28, 1), ()), types: (tf.uint8, tf.int64)>

In [23]:
# train = np.random.rand(300, 35, 7).astype(np.float32)
# label = np.random.rand(2, 7, 8, 9, 11).astype(np.float32)



# with strategy.scope():
    
#     # MODEL
#     ind = tf.keras.Input(shape=(4), dtype=tf.int64)       # (None, 4)

#     x = tf.keras.Input(shape=(35, 7), dtype=tf.float32)   # (None, 35, 7)
#     net = tf.keras.layers.Dense(11, activation='relu')(x)   # (None, 35, 11)
#     net = tf.reduce_max(net, axis=1, name='maxpool')          # (None, 11)
    
#     dense_shape = np.array([2, 7, 8, 9, 11], np.int64)
#     y = tf.scatter_nd(ind, net, dense_shape)
    
#     model = tf.keras.Model(inputs=[x, ind], outputs=y)

#     model.compile(loss='sparse_categorical_crossentropy',
#                   optimizer=tf.keras.optimizers.Adam())

# model.fit(train_dataset, epochs=12)

# DATASET practice

In [26]:
# import itertools
# tf.compat.v1.enable_eager_execution()

def gen():
    while True:
        n = np.random.randint(1, 7)
        yield ((np.random.rand(n, 3).astype(np.float32),   # (n, 3)
                np.random.rand(n, 3).astype(np.int64)),    # (n, 3)
               np.random.rand(11, 2).astype(np.float32))    # (11, 2)
     
        
ds = tf.data.Dataset.from_generator(
    gen, ((tf.float32, tf.int64), tf.float32), 
    ((tf.TensorShape([None, 3]), tf.TensorShape([None, 3])), tf.TensorShape([11, 2])) )

for value in ds.take(2):
    print(value[0][0].shape, value[0][1].shape, value[1].shape)
    
    
# (1, array([1]))
# (2, array([1, 1]))

(1, 3) (1, 3) (11, 2)
(4, 3) (4, 3) (11, 2)


In [27]:
dist_dataset = strategy.experimental_distribute_dataset(ds)

In [28]:
iterator = iter(dist_dataset)

In [29]:
for _ in range(3):
    aa = next(iterator)
    print(aa[0][0].shape, aa[0][1].shape, aa[1].shape)

(4, 3) (4, 3) (11, 2)
(1, 3) (1, 3) (11, 2)
(4, 3) (4, 3) (11, 2)
